In [1]:
import os
import numpy as np
import cv2
from skimage.measure import label 
import pydicom
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import torch
from torchvision import transforms as T   
import csv
from PIL import Image

In [6]:
def denorm(x):
    """Convert the range from [-1, 1] to [0, 1]."""
    out = (x + 1) / 2
    return out

def renormalize(ary):
    """Convert the range from [0, 1] to [-1000, 1000]."""
    re_ary = (ary*(1000. + 1000.)) - 1000.
    return re_ary


def _preprocess_cbct_ct(dicom):
    hu_data = dicom.pixel_array.astype(np.float32) * dicom.RescaleSlope + dicom.RescaleIntercept
    clip_hu = np.clip(hu_data, -1000, 1000)
    nor_hu = (clip_hu + 1000.) / (1000. + 1000.)
    return nor_hu

def _preprocess_mri(dicom):
    pixel_array = dicom.pixel_array.astype(np.float32) * dicom.RescaleSlope + dicom.RescaleIntercept
    clip_in = np.clip(pixel_array, 0, 1500)
    nor_hu = clip_in / (1500)
    return nor_hu

def read_dicom_series(dicom_directory):
    dicom_files = [os.path.join(dicom_directory, filename) for filename in os.listdir(dicom_directory) if filename.endswith('.dcm')]
    dicom_files.sort()
    dicom_slices = [pydicom.read_file(file_path) for file_path in dicom_files]
    if 'CT' in dicom_directory:
        image_array = np.stack([_preprocess_cbct_ct(slice) for slice in dicom_slices])
        return image_array
    if 'MRI' in dicom_directory:
        image_array = np.stack([_preprocess_mri(slice) for slice in dicom_slices])
        return image_array

def calculate_dsc(image_gt, image_pred):
    # Define thresholds for segmentation
    bone_threshold = 250
    soft_tissue_lower = -200
    soft_tissue_upper = 250
    air_threshold = -200
    
    # Perform segmentation based on thresholds
    seg_bone_gt = (image_gt >= bone_threshold).astype(int)
    seg_bone_pred = (image_pred >= bone_threshold).astype(int)
    
    seg_soft_tissue_gt = ((image_gt >= soft_tissue_lower) & (image_gt < soft_tissue_upper)).astype(int)
    seg_soft_tissue_pred = ((image_pred >= soft_tissue_lower) & (image_pred < soft_tissue_upper)).astype(int)
    
    seg_air_gt = (image_gt < air_threshold).astype(int)
    seg_air_pred = (image_pred < air_threshold).astype(int)
    
    # Calculate DSC for each segment
    def calculate_single_dsc(seg_gt, seg_pred):
        intersection = np.logical_and(seg_gt, seg_pred).sum()
        union = seg_gt.sum() + seg_pred.sum()
        dsc = (2 * intersection) / union if union != 0 else 1.0
        return dsc
    
    dsc_bone = calculate_single_dsc(seg_bone_gt, seg_bone_pred)
    dsc_soft_tissue = calculate_single_dsc(seg_soft_tissue_gt, seg_soft_tissue_pred)
    dsc_air = calculate_single_dsc(seg_air_gt, seg_air_pred)
    
    return dsc_bone, dsc_soft_tissue, dsc_air

def segment_bone_soft_air(image_pred):
    # Define thresholds for segmentation
    bone_threshold = 250
    soft_tissue_lower = -200
    soft_tissue_upper = 250
    air_threshold = -200
    # Create seg mask
    seg_bone_mask = (image_pred >= bone_threshold).astype(int)
    seg_soft_tissue_mask = ((image_pred >= soft_tissue_lower) & (image_pred < soft_tissue_upper)).astype(int)
    seg_air_mask = (image_pred < air_threshold).astype(int)
    # Apply mask
    seg_bone = image_pred * seg_bone_mask
    seg_soft_tissue = image_pred * seg_soft_tissue_mask
    seg_air_mask = image_pred * seg_air_mask
    return seg_bone, seg_soft_tissue, seg_air_mask

In [7]:
# Load Test data
test_dir = '/home/chanon/projects/Boo_Thesis/My_SuperStarGAN/data/test'
report_dir = '/home/chanon/projects/Boo_Thesis/My_SuperStarGAN/report only CBCT proceeding'

if not os.path.exists(report_dir):
    os.makedirs(report_dir)

CT_test = []
CBCT_test = []

for pt in os.listdir(test_dir):
    if pt != ".DS_Store":
        case_pt = os.path.join(test_dir, pt)
        pt_CT_test = read_dicom_series(os.path.join(case_pt, 'CT'))
        pt_CBCT_test = read_dicom_series(os.path.join(case_pt, 'CBCT'))
        CT_test.append(pt_CT_test)
        CBCT_test.append(pt_CBCT_test)

CT_test = np.stack(CT_test, axis=0)
CBCT_test = np.stack(CBCT_test, axis=0)

# Create a CSV file for recording CBCT and MRI MAE values in test
csv_file_s = os.path.join(report_dir, 'Test_summary.csv')
csv_file_c = os.path.join(report_dir, 'Test_case_result.csv')
csv_file_r = os.path.join(report_dir, 'Test_raw.csv')

In [8]:
all_mae_CBCT = []
all_mae_bone_CBCT = []
all_mae_soft_tissue_CBCT = []
all_mae_air_CBCT = []
all_psnr_CBCT = []
all_ssim_CBCT = []
all_dsc_bone_CBCT = []
all_dsc_soft_tissue_CBCT = []
all_dsc_air_CBCT = []

transform = []
transform.append(T.Resize((512,512)))
transform.append(T.ToTensor())
transform.append(T.Lambda(lambda x: x.expand(3, -1, -1)))
transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
transform = T.Compose(transform)

for c in range(CBCT_test.shape[0]):
    all_mae_CBCT_case = []
    all_mae_bone_CBCT_case = []
    all_mae_soft_tissue_CBCT_case = []
    all_mae_air_CBCT_case = []
    all_psnr_CBCT_case = []
    all_ssim_CBCT_case = []
    all_dsc_bone_CBCT_case = []
    all_dsc_soft_tissue_CBCT_case = []
    all_dsc_air_CBCT_case = []
               
    for s in range(CBCT_test.shape[1]):
        #=========================================================================================#
        # Preprocess image.
        #=========================================================================================#
        CBCT_PIL_array = Image.fromarray(CBCT_test[c][s])
        CBCT_pixel_array = transform(CBCT_PIL_array)
        CBCT_image_sample = torch.tensor(CBCT_pixel_array, dtype=torch.float32)
        CT_PIL_array = Image.fromarray(CT_test[c][s])
        CT_pixel_array = transform(CT_PIL_array)
        CT_image_sample = torch.tensor(CT_pixel_array, dtype=torch.float32)
        #=========================================================================================#
        # Denorm and Renorm.
        #=========================================================================================#
        CBCT_de = denorm(np.array(CBCT_image_sample))
        CBCT_hu = renormalize(CBCT_de)[0]
        
        CT_de = denorm(np.array(CT_image_sample))
        CT_hu = renormalize(CT_de)[0]
        #=========================================================================================#
        # Calculate.
        #=========================================================================================#
        mae = np.mean(np.abs(CT_hu - CBCT_hu))
        psnr_value = psnr(CT_hu , CBCT_hu, data_range= CT_hu.max() - CT_hu.min())
        ssim_value = ssim(CT_hu , CBCT_hu, data_range= CT_hu.max() - CT_hu.min())
        dsc_bone, dsc_soft_tissue, dsc_air = calculate_dsc(CT_hu , CBCT_hu)
        seg_bone_sCT, seg_soft_tissue_sCT, seg_air_sCT = segment_bone_soft_air(CBCT_hu)
        seg_bone_CT, seg_soft_tissue_CT, seg_air_CT = segment_bone_soft_air(CT_hu)
        mae_bone = np.mean(np.abs(seg_bone_CT - seg_bone_sCT))
        mae_soft_tissue = np.mean(np.abs(seg_soft_tissue_CT - seg_soft_tissue_sCT))
        mae_air = np.mean(np.abs(seg_air_CT - seg_air_sCT))

        all_dsc_bone_CBCT_case.append(dsc_bone)
        all_dsc_soft_tissue_CBCT_case.append(dsc_soft_tissue)
        all_dsc_air_CBCT_case.append(dsc_air)
        all_psnr_CBCT_case.append(psnr_value)
        all_ssim_CBCT_case.append(ssim_value)
        all_mae_CBCT_case.append(mae)
        all_mae_bone_CBCT_case.append(mae_bone)
        all_mae_soft_tissue_CBCT_case.append(mae_soft_tissue)
        all_mae_air_CBCT_case.append(mae_air)
        #=========================================================================================#
    all_dsc_bone_CBCT.append(all_dsc_bone_CBCT_case)
    all_dsc_soft_tissue_CBCT.append(all_dsc_soft_tissue_CBCT_case)
    all_dsc_air_CBCT.append(all_dsc_air_CBCT_case)
    all_psnr_CBCT.append(all_psnr_CBCT_case)
    all_ssim_CBCT.append(all_ssim_CBCT_case)
    all_mae_CBCT.append(all_mae_CBCT_case)
    all_mae_bone_CBCT.append(all_mae_bone_CBCT_case)
    all_mae_soft_tissue_CBCT.append(all_mae_soft_tissue_CBCT_case)
    all_mae_air_CBCT.append(all_mae_air_CBCT_case)
    #=========================================================================================#
    with open(csv_file_r, mode='a', newline='') as file:
            writer = csv.writer(file)
            for i in range(len(all_mae_CBCT_case)):
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'MAE', all_mae_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'SSIM', all_ssim_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'PSNR', all_psnr_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'MAE Soft tissue', all_mae_soft_tissue_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'MAE Bone', all_mae_bone_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'MAE Air', all_mae_air_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'DSC Soft tissue', all_dsc_soft_tissue_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'DSC Bone', all_dsc_bone_CBCT_case[i]])
                writer.writerow(['Case_{}'.format(c+1), f'Slice_{i+1}', 'DSC Air', all_dsc_air_CBCT_case[i]])
    #=========================================================================================#
    with open(csv_file_c, mode='a', newline='') as file:
        writer = csv.writer(file)
        case = ['Case_{}'.format(c+1), 'Case_{}'.format(c+1), 'Case_{}'.format(c+1), 'Case_{}'.format(c+1),
                'Case_{}'.format(c+1), 'Case_{}'.format(c+1), 'Case_{}'.format(c+1), 'Case_{}'.format(c+1), 'Case_{}'.format(c+1)]
        metic = ['MAE', 'MAE Bone', 'MAE Soft tissue', 'MAE Air',
              'SSIM', 'PSNR', 'DSC Bone', 'DSC Soft tissue', 'DSC Air']
        type = ['CBCT', 'CBCT', 'CBCT', 'CBCT',
                'CBCT', 'CBCT', 'CBCT', 'CBCT', 'CBCT']
        value = ['{} + {}'.format(round(np.mean(all_mae_CBCT_case),2), round(np.std(all_mae_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_ssim_CBCT_case),2), round(np.std(all_ssim_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_psnr_CBCT_case),2), round(np.std(all_psnr_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_mae_soft_tissue_CBCT_case),2), round(np.std(all_mae_soft_tissue_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_mae_bone_CBCT_case),2), round(np.std(all_mae_bone_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_mae_air_CBCT_case),2), round(np.std(all_mae_air_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_dsc_soft_tissue_CBCT_case),2), round(np.std(all_dsc_soft_tissue_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_dsc_bone_CBCT_case),2), round(np.std(all_dsc_bone_CBCT_case),2)),
                '{} + {}'.format(round(np.mean(all_dsc_air_CBCT_case),2), round(np.std(all_dsc_air_CBCT_case),2)),
                ]
        for i in range(len(metic)):
            writer.writerow([case[i], metic[i], type[i], value[i]])
#=========================================================================================#

print('MAE of CBCT = {}({}) HU'.format(round(np.mean(all_mae_CBCT),2), round(np.std(all_mae_CBCT),2)))
print('SSIM of CBCT = {}({}) A.U.'.format(round(np.mean(all_ssim_CBCT),2), round(np.std(all_ssim_CBCT),2)))
print('PSNR of CBCT = {}({}) dB'.format(round(np.mean(all_psnr_CBCT),2), round(np.std(all_psnr_CBCT),2)))
print('MAE Soft tissue CBCT = {} ({})'.format(round(np.mean(all_mae_soft_tissue_CBCT),2), round(np.std(all_mae_soft_tissue_CBCT),2)))
print('MAE Bone CBCT = {} ({})'.format(round(np.mean(all_mae_bone_CBCT),2), round(np.std(all_mae_bone_CBCT),2)))
print('MAE Air CBCT = {} ({})'.format(round(np.mean(all_mae_air_CBCT),2), round(np.std(all_mae_air_CBCT),2)))
print('DSC Soft tissue of CBCT = {}({})'.format(round(np.mean(all_dsc_soft_tissue_CBCT),2), round(np.std(all_dsc_soft_tissue_CBCT),2)))
print('DSC Bone of CBCT = {}({})'.format(round(np.mean(all_dsc_bone_CBCT),2), round(np.std(all_dsc_bone_CBCT),2)))
print('DSC Air of CBCT = {}({})'.format(round(np.mean(all_dsc_air_CBCT),2), round(np.std(all_dsc_air_CBCT),2)))

# Write CBCT and MRI MAE values to the CSV file
with open(csv_file_s, mode='a', newline='') as file:
        writer = csv.writer(file)
        metic = ['MAE', 'SSIM', 'PSNR',
                'MAE Soft tissue', 'MAE Bone', 'MAE Air',
                'DSC Soft tissue', 'DSC Bone','DSC Air']
        
        value_CBCT = ['{} + {}'.format(round(np.mean(all_mae_CBCT),2), round(np.std(all_mae_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_ssim_CBCT),2), round(np.std(all_ssim_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_psnr_CBCT),2), round(np.std(all_psnr_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_mae_soft_tissue_CBCT),2), round(np.std(all_mae_soft_tissue_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_mae_bone_CBCT),2), round(np.std(all_mae_bone_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_mae_air_CBCT),2), round(np.std(all_mae_air_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_dsc_soft_tissue_CBCT),2), round(np.std(all_dsc_soft_tissue_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_dsc_bone_CBCT),2), round(np.std(all_dsc_bone_CBCT),2)),
                      '{} + {}'.format(round(np.mean(all_dsc_air_CBCT),2), round(np.std(all_dsc_air_CBCT),2))
                      ]
        
        writer.writerow(['', metic[0], metic[1], metic[2]])
        writer.writerow(['CBCT', value_CBCT[0], value_CBCT[1], value_CBCT[2]])
        writer.writerow(['',metic[3], metic[4], metic[5]])
        writer.writerow(['CBCT',value_CBCT[3], value_CBCT[4], value_CBCT[5]])
        writer.writerow(['',metic[6], metic[7], metic[8]])
        writer.writerow(['CBCT',value_CBCT[6], value_CBCT[7], value_CBCT[8]])

/tmp/ipykernel_112598/3542524218.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  CBCT_image_sample = torch.tensor(CBCT_pixel_array, dtype=torch.float32)
/tmp/ipykernel_112598/3542524218.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  CT_image_sample = torch.tensor(CT_pixel_array, dtype=torch.float32)


MAE of CBCT sCT = 25.657039642333984(40.78664016723633) HU
SSIM of CBCT sCT = 0.8589843634580034(0.12242615393362599) A.U.
PSNR of CBCT sCT = 27.388020522525906(9.537000196218855) dB
DSC Bone of CBCT sCT = 0.7884159274228825(0.4592321621609294)
DSC Soft tissue of CBCT sCT = 0.9447798528088834(0.1386324464362656)
DSC Air of CBCT sCT = 0.9874906856311993(0.03327386799339771)
